### 0. 事前準備

以下のデータを取得して、このノートブックと同じディレクトリにある`data`ディレクトリに配置します。

__120 years of Olympic history: athletes and results__  
  URL:  
  ・https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results  
  DATA:  
  ・athlete_events.csv

---

__World cities database__  
  URL:  
    ・https://www.kaggle.com/juanmah/world-cities  
  DATA:  
    ・worldcities.csv

In [ ]:
%matplotlib inline

import folium
import matplotlib.pyplot as plt
import pandas as pd
import requests
import seaborn as sns

### 1. データの概要を把握
まずデータの概要を掴んでみましょう。

オリンピック選手データをロード。

In [ ]:
athlete_events_df = pd.read_csv('data/athlete_events.csv')

print(f"{len(athlete_events_df):,}件")

データの中身を確認。

不明な属性があれば、データ取得元で確認します。

In [ ]:
athlete_events_df.head()

数値属性の集計情報を確認。

どんな特徴が読み取れますか？

In [ ]:
athlete_events_df.describe()

ヒストグラムでデータを表示。

どんな特徴が読み取れますか？

In [ ]:
athlete_events_df.drop(columns=['ID']).hist(bins=50, figsize=(20, 12)); # セミコロンでテキスト出力を抑える

個々の属性の分布だけでなく、属性間の関係を見たい場合は散布図行列で表示します。

どんな特徴が読み取れますか？

In [ ]:
sns.pairplot(athlete_events_df.drop(columns=['ID']).sample(frac=0.5))

### 2. 地図を使ってデータを把握
地理情報がある場合は、地図上にデータを表示させてみましょう。

ここでは、開催都市を以下のルールに沿って地図上に表示してみます。
- 夏は赤系、冬は青系の円を配置。
- 参加選手数に合わせて円を大きくする。
- 複数回開催している場合は色分けする。

まずはデータを都市、ゲームでグループ分けします。


In [ ]:
city_games_df = athlete_events_df.groupby(['City', 'Year', 'Games']).agg(number_of_athletes=('ID', 'count'))
city_games_df.reset_index(inplace=True)

city_games_df.head(10)

色分け用に`season`列（夏季か冬季か？）を追加します。

In [ ]:
city_games_df['season'] = city_games_df['Games'].str.split().str[1]

city_games_df.head(10)

複数回開催している都市があるので、何回目の開催であるかを示す`number_of_times`列も追加します。

In [ ]:
city_games_df['number_of_times'] = city_games_df.groupby(['City', 'season']).cumcount() + 1

city_games_df.head(10)

世界の都市の位置情報データをロード。

In [ ]:
worldcities_df = pd.read_csv('data/worldcities.csv')

len(worldcities_df)

データを確認。

In [ ]:
worldcities_df.head(10)

`city_ascii`が英語名のようなので、`city_ascii`を都市名として利用。

同名の都市がないかチェックします。

In [ ]:
worldcities_df.duplicated(subset='city_ascii').sum()

都市名の重複が相当ある模様。

どんな重複があるのか確認してみます。

In [ ]:
worldcities_df[worldcities_df.duplicated(subset='city_ascii')].sort_values('city_ascii')

とりあえず機械的に、都市名が重複している場合は人口数の多いものだけ残すようにします。

具体的には都市名と人口の昇順に並べ替え、各重複都市名のうち最後の行、つまり一番人口が多いものだけを残します。

In [ ]:
worldcities_df.sort_values(['city_ascii', 'population'], inplace=True)

worldcities_df.drop_duplicates(subset='city_ascii', keep='last', inplace=True)

もう一度重複数を確認。

In [ ]:
worldcities_df.duplicated(subset='city_ascii').sum()

必要な列のみ残します。

In [ ]:
worldcities_df = worldcities_df[['city_ascii', 'lat', 'lng']]

都市名をキーとしてデータフレームを結合。

In [ ]:
city_game_coords_df = pd.merge(city_games_df,
                               worldcities_df,
                               left_on=['City'], 
                               right_on=['city_ascii'], 
                               how='left')

city_game_coords_df.head(10)

緯度経度が入っていないものをチェックします。

In [ ]:
city_game_coords_df[city_game_coords_df['lat'].isna()]

数が少ないので、個別対応します。
- Antwerpen → Antwerp
- Athina → Athens 
- Chamonix → Chamonix-Mont-Blanc
- Garmisch-Partenkirchen → Garmisch-Partenkirchen (47.492, 11.0931)
- Moskva → Moscow
- Sankt Moritz → St. Moritz (46.578, 9.8353)
- Squaw Valley → Palisades Tahoe (39.208, -120.2132)
- Torino → Turin

In [ ]:
city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Antwerpen"),
    ['city_ascii', 'lat', 'lng']] = worldcities_df.query('city_ascii == "Antwerp"').values

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Athina"),
    ['city_ascii', 'lat', 'lng']] = worldcities_df.query('city_ascii == "Athens"').values

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Chamonix"),
    ['city_ascii', 'lat', 'lng']] = worldcities_df.query('city_ascii == "Chamonix-Mont-Blanc"').values

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Garmisch-Partenkirchen"),
    ['city_ascii', 'lat', 'lng']] = ['Garmisch-Partenkirchen', 47.492, 11.0931]

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Moskva"),
    ['city_ascii', 'lat', 'lng']] = worldcities_df.query('city_ascii == "Moscow"').values

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Sankt Moritz"),
    ['city_ascii', 'lat', 'lng']] = ['St. Moritz', 46.578, 9.8353]

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Squaw Valley"),
    ['city_ascii', 'lat', 'lng']] = ['Palisades Tahoe', 39.208, -120.2132]

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Torino"),
    ['city_ascii', 'lat', 'lng']] = worldcities_df.query('city_ascii == "Turin"').values

もう一度、緯度経度が入っていないものをチェックします。

In [ ]:
city_game_coords_df[city_game_coords_df['lat'].isna()]

都市別開催数の最大値を確認。

In [ ]:
city_game_coords_df['number_of_times'].max()

円の色を用意します。

In [ ]:
number_of_times_max = city_game_coords_df['number_of_times'].max()

# 夏季用
color_reds = sns.color_palette("Reds_r", number_of_times_max).as_hex()
# 冬季用
color_blues = sns.color_palette("Blues_r", number_of_times_max).as_hex()

# 色の確認
sns.palplot(color_reds)
sns.palplot(color_blues)

データを地図上に表示します。

どんなことが読み取れますか？

In [ ]:
m = folium.Map(location=[0,0], zoom_start=2.4)

# 古い円をクリックできるよう、新しいものから順に重ねて描画
for index, row in city_game_coords_df.sort_values('Year', ascending=False).iterrows():
    location = (row['lat'], row['lng'])
    radius = row['number_of_athletes'] * 10
    color = color_reds[row['number_of_times'] - 1] if row['season'] == 'Summer' \
                                                   else color_blues[row['number_of_times'] - 1]

    folium.Circle(location=location,
                  radius=radius,
                  color=color,
                  fill_color=color,
                  weight=1.5,
                  popup=f"{row['City']}\n{row['Year']}\n{row['season']}").add_to(m)
m

### 3. 少しだけ掘り下げた分析をしてみる
細かいところは気にせず、大まかな流れを理解すればOKです。

日本の年代別・種類別メダル数がどのように推移しているかを分析します。

In [ ]:
# 日本選手に限定
athlete_jpn_df = athlete_events_df.query('NOC == "JPN"')

# 年、メダルでグループ分け
medal_jpn_df = athlete_jpn_df.groupby(['Year', 'Medal']).agg({'ID':'count'}).reset_index()

medal_jpn_df.head()

折れ線グラフで表示します。

In [ ]:
# 銅の色はbrownで代用
sns.lineplot(data=medal_jpn_df,
             x='Year',
             y='ID',
             hue='Medal',
             hue_order=["Gold", "Silver", "Bronze"],
             palette=["gold", "silver", "brown"])

### 4. 地図上にデータを表示するエクササイズ
JR東日本または東京メトロの各駅の乗降人員データをウェブ検索して取得し、地図上にデータを表示してみましょう。

WEBページ上のデータの取得方法:  
`pd.read_html()`に引数としてページのURLを指定すると、ページ上のすべての`table`タグのデータをデータフレームのリストとして取得します。  
エラーとなってデータフレームが取得できない場合、以下のようにHTMLソースを引数に渡します。

また、データフレームの行または列が階層構造になってしまう場合には`pandas.MultiIndex.droplevel()`を利用して階層構造を解除します。

参考）  
__pandas.MultiIndex.droplevel__  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.droplevel.html

In [ ]:
# HTMLソースを取得
response = requests.get('https://en.wikipedia.org/wiki/List_of_highest-grossing_films')
# 文字化けする場合は設定
response.encoding = response.apparent_encoding
# データフレームに変換
dfs = pd.read_html(response.text)
# 任意のデータフレームを選択
dfs[1]